Things to do: sentiment analysis of most recent 100 tweets of various news orgs

Initialize lots of empty lists

(build a function?)

* For one news org:
* Pull 100 last tweets (or pull 1 at a time, 100 times?)
** See 1-Solved-StuGetRelief for looping through 100
* Run each through Vader
* Collect: text, date, compound, pos, neg, neut; add to respective lists
* Average compounds for each news org; add to compound average list
* Clear lists


DF of lists

PLOT 1
* Actually 5 plots
* ?? How to number Tweets Ago? Need a column for this? Or use Pandas index?)*
* Plot compounds


PLOT 2
* Zip News org keys with compound average list into dict
* Create DF

* Build data frame of lists


Export to CSV, Save plots as PNG


In [17]:
# Import Dependencies

import tweepy
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Twitter API Keys
from config import (consumer_key, 
                    consumer_secret, 
                    access_token, 
                    access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [18]:
# Target User Account
target_user = "@CNN"

# Counter
counter = 1

# Loop through 5 pages of tweets (total 100 tweets)
for x in range(5):

    # Get all tweets from home feed
    public_tweets = api.user_timeline(target_user, page=x)

    # Loop through all tweets
    for tweet in public_tweets:

        # Print Tweets
        print(f'Tweet {counter}: {tweet["text"]}')

        # Add to Counter
        counter = counter + 1

Tweet 1: Florida boy wanted by police in connection with suspected death of his grandmother is caught near Canadian border,… https://t.co/WPGWpJdvPJ
Tweet 2: 70% of Americans — with majorities in both parties — think that Puerto Rico is not getting the help it needs in the… https://t.co/zSkODmjpiQ
Tweet 3: A federal judge has permanently blocked President Trump’s executive order to cut funding from so-called “sanctuary… https://t.co/2PGGGrTikb
Tweet 4: Meet 'Oumuamua, a cigar-shaped asteroid that is the first observed interstellar visitor to our solar system… https://t.co/cOkDsbIGQM
Tweet 5: A paper claims a sugar-industry-funded study linked sugar to high blood cholesterol and cancer in rats. The study i… https://t.co/fbGEtNrX3Z
Tweet 6: Longtime actor Rance Howard, father of director Ron Howard, has died at age 89, Ron Howard said in a tweet… https://t.co/fAVXeLcASR
Tweet 7: Puerto Rico says only 55 died from Hurricane Maria, but funeral homes claim the number should be nine times hi

Tweet 61: The White House vehemently has defended President Trump's decision to appoint Mick Mulvaney as the acting director… https://t.co/qvPApFOhUJ
Tweet 62: Turkey's foreign minister says President Trump agreed to stop arming a Kurdish militia the Turkish government consi… https://t.co/5QXj7qI6MW
Tweet 63: The US Navy has identified three sailors who went missing after their plane crashed into the Philippine Sea… https://t.co/bP6o2e1tze
Tweet 64: President Trump named an interim head of a consumer watchdog agency hours after the outgoing director picked his ow… https://t.co/a4JYiVPUG8
Tweet 65: In 2017, the world has made one thing clear: Sexual harassment is everywhere. Here is how the issue stands across t… https://t.co/rmvI65ZQb3
Tweet 66: The happiest city in the US is Boulder, Colorado. Here's the full top 10: https://t.co/eoWFSq3dTW https://t.co/my5iRlqAt0
Tweet 67: His wife is representing USA in the world’s most prestigious transgender pageant. @lisaling takes us on their jo

In [19]:
# Target User Accounts
target_users = ("@BBC", "@CBS", "@CNN", "@FoxNews", "@NYT")

# Variable for holding sentiments
sentiments = []

#Variables for holding average Vader compound scores

# Loop through all target users
for target in target_users:

    # Variable for holding the oldest tweet
    #oldest_tweet = None

    # Variables for holding sentiments
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []

    # Loop through 5 pages of tweets (total 100 tweets)
    for x in range(5):

        # Get all tweets from home feed
        public_tweets = api.user_timeline(target)

        # Loop through all tweets 
        for tweet in public_tweets:

            # Print Tweets
            #print("Tweet %s: %s" % (counter, tweet["text"]))

            # Run Vader Analysis on each tweet
            results = analyzer.polarity_scores(tweet["text"])
            compound = results["compound"]
            pos = results["pos"]
            neu = results["neu"]
            neg = results["neg"]
            #tweets_ago = counter
        
            # Add sentiments for each tweet into an array
            sentiments.append({"Tweet": tweet["text"],
                               "News Org": target,
                            "Date": tweet["created_at"], 
                           "Compound": compound,
                           "Positive": pos,
                           "Negative": neu,
                           "Neutral": neg})

In [20]:
# Convert sentiments to DataFrame
sentiments_pd = pd.DataFrame.from_dict(sentiments)

# Configure dataframe column order and display
sentiments_pd = sentiments_pd[['News Org',
                         'Date',
                         'Tweet',
                         'Compound',
                         'Positive',
                         'Neutral',
                         'Negative']]

sentiments_pd

,News Org,Date,Tweet,Compound,Positive,Neutral,Negative
0,@BBC,Sat Nov 25 20:35:04 +0000 2017,The frozen bodies of 15 men are discovered on ...,-0.7845,0.000,0.266,0.734
1,@BBC,Sat Nov 25 19:53:03 +0000 2017,"50 years since his murder, celebrate the wit, ...",-0.2500,0.140,0.178,0.682
2,@BBC,Sat Nov 25 19:33:02 +0000 2017,Re-rewind! 🎤 🙌 Who else wants to watch @CraigD...,0.0000,0.000,0.000,1.000
3,@BBC,Sat Nov 25 18:30:06 +0000 2017,The London mum using her maternity leave to tr...,0.2960,0.106,0.053,0.841
4,@BBC,Sat Nov 25 18:14:06 +0000 2017,Claw blimey! 🦀 Here's why thousands of spider ...,0.0000,0.000,0.000,1.000
5,@BBC,Sat Nov 25 17:30:42 +0000 2017,RT @BBCMOTD: Chaos at Norwich...\n\nA 'local b...,-0.1027,0.150,0.168,0.682
6,@BBC,Sat Nov 25 17:30:08 +0000 2017,The @WorldPressPhoto's Noortje Gorter discusse...,0.0000,0.000,0.000,1.000
7,@BBC,Sat Nov 25 17:25:18 +0000 2017,RT @bbcstrictly: It's #Strictly Saturday! We a...,0.0000,0.000,0.000,1.000
8,@BBC,Sat Nov 25 17:24:41 +0000 2017,RT @BBCR1: Here's 14 of the best Live Lounge p...,0.7964,0.283,0.000,0.717
9,@BBC,Sat Nov 25 17:24:30 +0000 2017,RT @BBCiPlayer: **** GEO block to local people...,-0.4404,0.000,0.225,0.775


In [21]:
average_compound = sentiments_pd.groupby("News Org").mean()["Compound"]

compounds_df = pd.DataFrame({"Average Compound Score": average_compound})
compounds_df

,Average Compound Score
News Org,
@BBC,-0.005780
@CBS,0.410705
@CNN,-0.196995
@FoxNews,-0.036785
@NYT,-0.169215
